In [1]:
import spotipy
import pandas as pd
import numpy as np
from timeit import default_timer as timer
from datetime import timedelta
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from PIL import Image
from wordcloud import WordCloud
import random

In [2]:
from spotipy.oauth2 import SpotifyClientCredentials

client_id= 'CLIENT_ID'
client_secret = 'CLIENT_SECRET'
client_credentials_manager = SpotifyClientCredentials(
                                client_id = client_id, client_secret = client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
def format_time(seconds):
    minutes = seconds // 60
    seconds = seconds % 60
    if minutes > 0:
        return f"{minutes} minutes, {seconds} seconds"
    else:
        return f"{seconds} seconds"

In [4]:
start_time = timer()

# create empty lists to store data
artist_name = []
track_name = []
track_id = []
album_name = []
album_id = []
release_date = []
duration_ms = []
popularity = []
explicit = []
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
time_signature = []
featured_artists = []

# loop through results, using offset to get all tracks
for i in range(0, 1000, 50):
    track_results = sp.search(q='artist:ed sheeran', type='track', limit=50, offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
#         prevent live performances from entering the dataset
        if "live" not in t['name'].lower() and "live" not in t['album']['name'].lower():
            # get track details
            artist_name.append(t['artists'][0]['name'])
            track_name.append(t['name'])
            track_id.append(t['id'])
            album_name.append(t['album']['name'])
            album_id.append(t['album']['id'])
            release_date.append(t['album']['release_date'])
            popularity.append(t['popularity'])
            explicit.append(t['explicit'])

            # get audio features for track
            audio_features = sp.audio_features(t['id'])[0]
            danceability.append(audio_features['danceability'])
            energy.append(audio_features['energy'])
            key.append(audio_features['key'])
            loudness.append(audio_features['loudness'])
            mode.append(audio_features['mode'])
            speechiness.append(audio_features['speechiness'])
            acousticness.append(audio_features['acousticness'])
            instrumentalness.append(audio_features['instrumentalness'])
            liveness.append(audio_features['liveness'])
            valence.append(audio_features['valence'])
            tempo.append(audio_features['tempo'])
            time_signature.append(audio_features['time_signature'])

            # get featured artists
            if len(t['artists']) > 1:
                feat_artists = []
                for j in range(1, len(t['artists'])):
                    feat_artists.append(t['artists'][j]['name'])
                featured_artists.append(feat_artists)
            else:
                featured_artists.append([])

# create dataframe from lists
df = pd.DataFrame({
    'artist_name': artist_name,
    'track_name': track_name,
    'track_id': track_id,
    'album_name': album_name,
    'album_id': album_id,
    'release_date': release_date,
    'popularity': popularity,
    'explicit': explicit,
    'danceability': danceability,
    'energy': energy,
    'key': key,
    'loudness': loudness,
    'mode': mode,
    'speechiness': speechiness,
    'acousticness': acousticness,
    'instrumentalness': instrumentalness,
    'liveness': liveness,
    'valence': valence,
    'tempo': tempo,
    'time_signature': time_signature,
    'featured_artists': featured_artists
})

# convert release_date column to datetime format
df['release_date'] = pd.to_datetime(df['release_date'])

# extract month and year from release_date column
df['release_month'] = df['release_date'].dt.strftime('%B')
df['release_year'] = df['release_date'].dt.year

# store dataframe as csv file
df.to_csv('Ed Sheeran Spotify Tracks.csv', index=False)



end_time = timer()
elapsed_time = int(end_time - start_time)
print(f"Elapsed time: {format_time(elapsed_time)}")

Elapsed time: 2 minutes, 49 seconds


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978 entries, 0 to 977
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   artist_name       978 non-null    object        
 1   track_name        978 non-null    object        
 2   track_id          978 non-null    object        
 3   album_name        978 non-null    object        
 4   album_id          978 non-null    object        
 5   release_date      978 non-null    datetime64[ns]
 6   popularity        978 non-null    int64         
 7   explicit          978 non-null    bool          
 8   danceability      978 non-null    float64       
 9   energy            978 non-null    float64       
 10  key               978 non-null    int64         
 11  loudness          978 non-null    float64       
 12  mode              978 non-null    int64         
 13  speechiness       978 non-null    float64       
 14  acousticness      978 non-

In [6]:
df.head()

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists,release_month,release_year
0,Ed Sheeran,Shivers,50nfwKoDiSYg8zOCREWAm5,=,32iAEBstCjauDhyKpGjTuq,2021-10-29,83,False,0.788,0.859,...,0.0856,0.2810,0.000000,0.0424,0.822,141.020,4,[],October,2021
1,Ed Sheeran,Perfect,0tgVpDi06FyKpA1z0VMD4v,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,85,False,0.599,0.448,...,0.0232,0.1630,0.000000,0.1060,0.168,95.050,3,[],March,2017
2,Ed Sheeran,Bad Habits,3rmo8F54jFF8OgYsqTxm5d,=,32iAEBstCjauDhyKpGjTuq,2021-10-29,83,False,0.807,0.893,...,0.0347,0.0451,0.000028,0.3660,0.537,126.011,4,[],October,2021
3,Ed Sheeran,Tenerife Sea,1WTY0VL681yvPxvUuJ5GXY,Intimate Pop,65qTx2hiYqEvBu192EoV8H,2023-02-24,1,False,0.526,0.345,...,0.0374,0.7000,0.000011,0.1050,0.357,121.877,4,[],February,2023
4,Ed Sheeran,Perfect,4oAhMSYRMl1SAv02GbG46I,The Words - Soft 10s,2heLXJ9F1z8P8ypjyale5D,2023-02-21,1,False,0.598,0.448,...,0.0232,0.1620,0.000000,0.1060,0.168,95.009,3,[],February,2023
